In [1]:
import numpy as n
import numpy as np
import pandas as pd


In [3]:
MILANO = pd.read_csv('Comune-di-Milano-Pubblici-esercizi(in)-2.csv',sep=';',encoding='utf-8')

In [4]:
settore_col = "Settore storico pe"
tipo_col    = "þÿTipo esercizio storico pe"

df = MILANO.copy()

# 1) Normalizzo il testo del settore per cercare le parole chiave in modo robusto
df["settore_norm"] = df[settore_col].astype(str).str.upper().fillna("")

# 2) Definisco i pattern per ogni macro-categoria -------------------------

# 1) BAR
patterns_bar = [
    "BAR CAFFE",                  # bar caffe
    "CAFF",                   # bar caffÿ, bar-caffè e simili
    "BAR GASTRONOM",              # bar gastronomici
    "BIRRERIA",                   # birreria
    "SALE DA BALLO",              # sale da ballo, locali notturni
    "BAR",
    "DISCO",
    "LOCALI NOTTURNI",
    "SPACCIO BEVANDE ANALCOLICHE",
    "GIOC",
    "SOMMINISTRAZIONE",
    "WINE",                       # wine,birr.,pub enot.,caff.,the
    "PUB"
]

# 2) PIZZERIA
patterns_piz = [
    "PIZZERIA",
    "PIZZERIE E SIMILI"
]

# 3) RISTORANTE
patterns_rist = [
    "RISTORA", #inclueds ristorante and ristorazione
    "OSTERIA",
    "CUCINA",
    "TRATTORIA"
    # copre anche "RISTORANTE, TRATTORIA, OSTERIA", "TRTTORIA" ecc. per match parziale
]

# 4) GASTRONOMIA
patterns_gast = [
    "GENERE MERCEOL",                # genere merceol.autorizz.sanit.
    "PRODOTTI DI GASTRONOMIA",      # prodotti di gastronomia
    "PROD DI GASTRO",               # varianti abbreviate
    "TAVOLA FREDDA",
    "CIBI COTTI",
    "CIBI COTTI PRECONFEZIONATI",
    "MENSA",
    "TAVOLA CALDA",
    "TAV.CALDE,SELF SERVICE,FAST F", # tav.calde,self service,fast f.
    "SELF SERVICE",
    "FAST F"
]

# 5) GELATERIA
patterns_gel = [
    "BAR PASTIC",   # bar pasticc.gelat.crem.creper.
    "GELATERIA"
]



# 3) Creo i mask per ogni macro-categoria ---------------------------------

def build_mask(patterns):
    """Ritorna una mask booleana True se settore_norm contiene almeno uno dei pattern."""
    regex = "|".join(patterns)
    return df["settore_norm"].str.contains(regex, na=False)

mask_bar   = build_mask(patterns_bar)
mask_piz   = build_mask(patterns_piz)
mask_rist  = build_mask(patterns_rist)
mask_gast  = build_mask(patterns_gast)
mask_gel   = build_mask(patterns_gel)

# 4) Assegno le macro-categorie con la GERARCHIA voluta -------------------
#    RISTORANTE > PIZZERIA > BAR > GASTRONOMIA > GELATERIA
#    Quindi metto prima le categorie a priorità più bassa, e per ultime quelle a priorità più alta.

df["Tipo_macro"] = "ALTRO"   # default

# Priorità più bassa: GELATERIA
df.loc[mask_gel,  "Tipo_macro"] = "GELATERIA"

# Poi GASTRONOMIA
df.loc[mask_gast, "Tipo_macro"] = "GASTRONOMIA"

# Poi BAR
df.loc[mask_bar,  "Tipo_macro"] = "BAR"

# Poi PIZZERIA
df.loc[mask_piz,  "Tipo_macro"] = "PIZZERIA"

# Infine RISTORANTE (priorità MASSIMA: sovrascrive PIZZERIA/BAR/GASTRONOMIA/GELATERIA)
df.loc[mask_rist, "Tipo_macro"] = "RISTORANTE"

# 5) Copio il risultato su MILANO -----------------------------------------

# Creo una nuova colonna con la macro-categoria
MILANO["Tipo esercizio macro pe"] = df["Tipo_macro"]

# Se vuoi DAVVERO riscrivere la colonna storica (io terrei l'originale):
# MILANO[tipo_col] = MILANO["Tipo esercizio macro pe"]

# Controllo rapido delle frequenze
MILANO.groupby("Tipo esercizio macro pe").count()

,þÿTipo esercizio storico pe,Insegna,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,Forma commercio,Forma commercio prev,Forma vendita,Settore storico pe,Superficie somministrazione
Tipo esercizio macro pe,,,,,,,,,,,,,
ALTRO,6,2,21,21,21,21,21,21,6,6,6,0,17
BAR,3049,1961,3654,3654,3654,3577,3654,3654,2922,3012,3000,3654,3629
GASTRONOMIA,22,15,246,246,246,239,246,246,23,23,22,246,217
GELATERIA,3,0,3,3,3,3,3,3,2,2,2,3,3
PIZZERIA,181,109,246,246,246,237,246,246,172,177,175,246,245
RISTORANTE,2290,1407,2734,2734,2734,2671,2734,2734,2208,2287,2275,2734,2714


In [5]:
idx_mismatch_zd = MILANO["Tipo esercizio macro pe"] == "ALTRO"
idx_mismatch_zd

0       False
1       False
2       False
3       False
4       False
        ...  
6899    False
6900    False
6901    False
6902    False
6903    False
Name: Tipo esercizio macro pe, Length: 6904, dtype: bool

In [7]:
MILANO["Tipo esercizio macro pe"].isnull().sum()

0

In [8]:
MILANO["Insegna"] = MILANO["Insegna"].fillna("Unknown")
MILANO

,þÿTipo esercizio storico pe,Insegna,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,Forma commercio,Forma commercio prev,Forma vendita,Settore storico pe,Superficie somministrazione,Tipo esercizio macro pe
0,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),ALZ,NAVIGLIO GRANDE,12,5144,6,NaN,NaN,NaN,"Ristorante, trattoria, osteria;Genere Merceol....",83.0,RISTORANTE
1,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),ALZ,NAVIGLIO GRANDE,44,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,26.0,BAR
2,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),ALZ,NAVIGLIO GRANDE,48,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,58.0,BAR
3,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),ALZ,NAVIGLIO GRANDE,8,5144,6,NaN,NaN,NaN,"BAR CAFFÿý E SIMILI;Ristorante, trattoria, ost...",101.0,RISTORANTE
4,NaN,Unknown,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),ALZ,NAVIGLIO PAVESE,24,5161,6,NaN,NaN,NaN,Bar gastronomici e simili,51.0,BAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6899,"wine,birr.,pub enot.,caff.,the",bar cherry,VLE DORIA ANDREA N. 12 ; isolato:031; accesso:...,VLE,DORIA ANDREA,12,2230,2,solo somministrazione,somministrazione,misto,"Wine,birr.,pub enot.,caff.,the",59.0,BAR
6900,"wine,birr.,pub enot.,caff.,the",la balusa,VIA GARIGLIANO N. 5 ; isolato:277; accesso: ac...,VIA,GARIGLIANO,5,1134,9,solo somministrazione,somministrazione,misto,"Wine,birr.,pub enot.,caff.,the",40.0,BAR
6901,"wine,birr.,pub enot.,caff.,the",la champagnerie sas,VIA SOTTOCORNO PASQUALE N. 4 ; isolato:014; ac...,VIA,SOTTOCORNO PASQUALE,4,3152,4,solo somministrazione,somministrazione,misto,BAR CAFFÿý E SIMILI;Bar gastronomici e simili,53.0,BAR
6902,"wine,birr.,pub enot.,caff.,the",old rooster,VIA CASTROVILLARI N. 23 ; isolato:150; accesso...,VIA,CASTROVILLARI,23,6299,7,solo somministrazione,somministrazione,misto,"Wine,birr.,pub enot.,caff.,the",43.0,BAR


In [9]:
MILANO["Superficie somministrazione"] = pd.to_numeric(
    MILANO["Superficie somministrazione"],
    errors="coerce"
)

In [10]:
superficie_mean = MILANO["Superficie somministrazione"].mean()

MILANO["Superficie somministrazione"] = MILANO["Superficie somministrazione"].fillna(superficie_mean)
MILANO

,þÿTipo esercizio storico pe,Insegna,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,Forma commercio,Forma commercio prev,Forma vendita,Settore storico pe,Superficie somministrazione,Tipo esercizio macro pe
0,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),ALZ,NAVIGLIO GRANDE,12,5144,6,NaN,NaN,NaN,"Ristorante, trattoria, osteria;Genere Merceol....",83.0,RISTORANTE
1,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),ALZ,NAVIGLIO GRANDE,44,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,26.0,BAR
2,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),ALZ,NAVIGLIO GRANDE,48,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,58.0,BAR
3,NaN,Unknown,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),ALZ,NAVIGLIO GRANDE,8,5144,6,NaN,NaN,NaN,"BAR CAFFÿý E SIMILI;Ristorante, trattoria, ost...",101.0,RISTORANTE
4,NaN,Unknown,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),ALZ,NAVIGLIO PAVESE,24,5161,6,NaN,NaN,NaN,Bar gastronomici e simili,51.0,BAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6899,"wine,birr.,pub enot.,caff.,the",bar cherry,VLE DORIA ANDREA N. 12 ; isolato:031; accesso:...,VLE,DORIA ANDREA,12,2230,2,solo somministrazione,somministrazione,misto,"Wine,birr.,pub enot.,caff.,the",59.0,BAR
6900,"wine,birr.,pub enot.,caff.,the",la balusa,VIA GARIGLIANO N. 5 ; isolato:277; accesso: ac...,VIA,GARIGLIANO,5,1134,9,solo somministrazione,somministrazione,misto,"Wine,birr.,pub enot.,caff.,the",40.0,BAR
6901,"wine,birr.,pub enot.,caff.,the",la champagnerie sas,VIA SOTTOCORNO PASQUALE N. 4 ; isolato:014; ac...,VIA,SOTTOCORNO PASQUALE,4,3152,4,solo somministrazione,somministrazione,misto,BAR CAFFÿý E SIMILI;Bar gastronomici e simili,53.0,BAR
6902,"wine,birr.,pub enot.,caff.,the",old rooster,VIA CASTROVILLARI N. 23 ; isolato:150; accesso...,VIA,CASTROVILLARI,23,6299,7,solo somministrazione,somministrazione,misto,"Wine,birr.,pub enot.,caff.,the",43.0,BAR


Nelle celle abbiamo gestito i valori mancanti in modo guidato dai dati, sfruttando le macro-categorie dei pubblici esercizi (BAR, RISTORANTE, PIZZERIA, GASTRONOMIA, GELATERIA) e introducendo una soglia di affidabilità. Per riempire i NaN di “Forma commercio prev” abbiamo osservato, per ciascuna macro-categoria, quale valore tra “somministrazione” e “minuto” fosse il più frequente nei record non mancanti e abbiamo calcolato la sua confidenza come percentuale sul totale della macro-categoria; abbiamo imputato i NaN solo quando questa confidenza era almeno 80%, così da completare il dato solo nei casi in cui la relazione macro→prev risultava chiaramente dominante ed evitare imputazioni arbitrarie nelle categorie ambigue. Dopo questa imputazione, abbiamo riempito

In [22]:
import pandas as pd
import numpy as np

prev_col  = "Forma commercio prev"
macro_col = "Tipo esercizio macro pe"
soglia = 0.80

# Normalizzazione valori
MILANO[prev_col]  = MILANO[prev_col].astype(str).str.strip().str.lower().replace("nan", np.nan)
MILANO[macro_col] = MILANO[macro_col].astype(str).str.strip().str.upper().replace("nan", np.nan)

# ---- 1) Costruisco la regola macro -> prev dominante + confidenza ----
tmp = MILANO[[macro_col, prev_col]].dropna(subset=[macro_col, prev_col]).copy()

counts = (
    tmp.groupby([macro_col, prev_col])
       .size()
       .rename("n")
       .reset_index()
)

tot = counts.groupby(macro_col)["n"].sum().rename("tot").reset_index()
counts = counts.merge(tot, on=macro_col, how="left")
counts["conf"] = counts["n"] / counts["tot"]

best = (
    counts.sort_values(["conf", "n"], ascending=False)
          .drop_duplicates(subset=[macro_col])
          [[macro_col, prev_col, "n", "tot", "conf"]]
          .sort_values("conf", ascending=False)
)

# (Opzionale) Mostro quali macro superano la soglia
best_over = best[best["conf"] >= soglia].copy()
print(f"Macro-categorie con conf >= {soglia}: {len(best_over)} su {best[macro_col].nunique()}")
display(best_over)

# ---- 2) Imputazione (solo se conf >= soglia) ----
mode_map = dict(zip(best[macro_col], best[prev_col]))
conf_map = dict(zip(best[macro_col], best["conf"]))

prev_na_before = MILANO[prev_col].isna().sum()

MILANO["_prev_mode"] = MILANO[macro_col].map(mode_map)
MILANO["_prev_conf"] = MILANO[macro_col].map(conf_map)

mask_fill_prev = MILANO[prev_col].isna() & (MILANO["_prev_conf"] >= soglia)
imputable_rows = int(mask_fill_prev.sum())

MILANO.loc[mask_fill_prev, prev_col] = MILANO.loc[mask_fill_prev, "_prev_mode"]

prev_na_after = MILANO[prev_col].isna().sum()

# pulizia colonne temporanee
MILANO.drop(columns=["_prev_mode", "_prev_conf"], inplace=True)

# ---- 3) Output richiesto ----
print(f"NaN prev prima: {prev_na_before}")
print(f"Righe imputabili (conf >= soglia): {imputable_rows}")
print(f"NaN prev dopo: {prev_na_after}")


Macro-categorie con conf >= 0.8: 5 su 6


,Tipo esercizio macro pe,Forma commercio prev,n,tot,conf
5,GASTRONOMIA,somministrazione,245,246,0.995935
11,RISTORANTE,somministrazione,2720,2734,0.994879
9,PIZZERIA,somministrazione,243,246,0.987805
3,BAR,somministrazione,3554,3654,0.972633
1,ALTRO,somministrazione,20,21,0.952381


NaN prev prima: 1
Righe imputabili (conf >= soglia): 0
NaN prev dopo: 1


abbiamo riempito i valori mancanti di “Forma commercio” usando “Forma commercio prev” come variabile di supporto, perché la colonna “prev” contiene una codifica più semplice che risulta fortemente collegata a quella nuova. Prima abbiamo osserrvato quale valore di “Forma commercio” fosse più frequente per ciascun valore di “Forma commercio prev”; poi abbiamo imputato solo le righe in cui “Forma commercio” era NaN, assegnando il valore dominante associato al corrispondente “prev” (in pratica, “somministrazione” tende a mappare su “solo somministrazione” e “minuto” su “somministrazione/minuto”), evitando di riscrivere i valori già presenti e limitandoci a colmare i missing sulla base della relazione osservata nei dati.

In [32]:
soglia = 0.80
new_col  = macro_col

dist = pd.crosstab(MILANO[prev_col], MILANO[new_col], normalize="index")
new_mode = dist.idxmax(axis=1)   # mapping prev -> new dominante
conf     = dist.max(axis=1)      # confidenza del mapping

# mask: devo imputare solo dove new è NaN, prev è noto, e conf >= soglia
mask_fill_new = (
    MILANO[new_col].isna() &
    MILANO[prev_col].notna() &
    (MILANO[prev_col].map(conf) >= soglia)
)

print("NaN new prima:", MILANO[new_col].isna().sum())
print("Righe imputabili (conf >= soglia):", int(mask_fill_new.sum()))

MILANO.loc[mask_fill_new, new_col] = MILANO.loc[mask_fill_new, prev_col].map(new_mode)

print("NaN new dopo:", MILANO[new_col].isna().sum())


NaN new prima: 0
Righe imputabili (conf >= soglia): 0
NaN new dopo: 0


In [28]:
import pandas as pd
import numpy as np

vend_col  = "Forma vendita"
new_col   = "Forma commercio"
prev_col  = "Forma commercio prev"
macro_col = "Tipo esercizio macro pe"

# normalizzazione valori
for c in [vend_col, new_col, prev_col]:
    MILANO[c] = MILANO[c].astype(str).str.strip().str.lower().replace("nan", np.nan)
MILANO[macro_col] = MILANO[macro_col].astype(str).str.strip().str.upper().replace("nan", np.nan)

# 1) Macro -> forma vendita (percentuali)
display(pd.crosstab(MILANO[macro_col], MILANO[vend_col], normalize="index", dropna=False))

# 2) Forma commercio -> forma vendita (percentuali)
display(pd.crosstab(MILANO[new_col], MILANO[vend_col], normalize="index", dropna=False))

# 3) (Macro, Forma commercio) -> forma vendita (percentuali)
display(pd.crosstab([MILANO[macro_col], MILANO[new_col]], MILANO[vend_col], normalize="index", dropna=False).head(30))

# 4) (Macro, Forma commercio prev) -> forma vendita (percentuali)
display(pd.crosstab([MILANO[macro_col], MILANO[prev_col]], MILANO[vend_col], normalize="index", dropna=False).head(30))


Forma vendita,al banco,al tavolo,misto,self service,NaN
Tipo esercizio macro pe,,,,,
ALTRO,0.190476,0.000000,0.095238,0.000000,0.714286
BAR,0.486590,0.013136,0.319376,0.001916,0.178982
GASTRONOMIA,0.048780,0.008130,0.028455,0.004065,0.910569
GELATERIA,0.666667,0.000000,0.000000,0.000000,0.333333
PIZZERIA,0.178862,0.170732,0.349593,0.012195,0.288618
RISTORANTE,0.107169,0.363936,0.347476,0.013533,0.167886


Forma vendita,al banco,al tavolo,misto,self service,NaN
Forma commercio,,,,,
solo somministrazione,0.275383,0.169430,0.323385,0.007421,0.224380
somministrazione/minuto,0.682456,0.024561,0.287719,0.001754,0.003509
NaN,0.000000,0.000000,0.000000,0.000000,1.000000


Forma vendita                                    al banco  al tavolo  \
Tipo esercizio macro pe Forma commercio                                
ALTRO                   solo somministrazione    0.157895   0.000000   
                        somministrazione/minuto  0.500000   0.000000   
                        NaN                      0.000000   0.000000   
BAR                     solo somministrazione    0.445933   0.013711   
                        somministrazione/minuto  0.779775   0.008989   
                        NaN                      0.000000   0.000000   
GASTRONOMIA             solo somministrazione    0.044898   0.008163   
                        somministrazione/minuto  1.000000   0.000000   
                        NaN                      0.000000   0.000000   
GELATERIA               solo somministrazione    1.000000   0.000000   
                        somministrazione/minuto  1.000000   0.000000   
                        NaN                      0.000000   0.000000   
PIZZERIA                solo somministrazione    0.171806   0.185022   
                        somministrazione/minuto  0.263158   0.000000   
                        NaN                      0.000000   0.000000   
RISTORANTE              solo somministrazione    0.098404   0.374240   
                        somministrazione/minuto  0.333333   0.098039   
                        NaN                      0.000000   0.000000   

Forma vendita                                       misto  self service  \
Tipo esercizio macro pe Forma commercio                                   
ALTRO                   solo somministrazione    0.052632      0.000000   
                        somministrazione/minuto  0.500000      0.000000   
                        NaN                      0.000000      0.000000   
BAR                     solo somministrazione    0.335307      0.001870   
                        somministrazione/minuto  0.204494      0.002247   
                        NaN                      0.000000      0.000000   
GASTRONOMIA             solo somministrazione    0.028571      0.004082   
                        somministrazione/minuto  0.000000      0.000000   
                        NaN                      0.000000      0.000000   
GELATERIA               solo somministrazione    0.000000      0.000000   
                        somministrazione/minuto  0.000000      0.000000   
                        NaN                      0.000000      0.000000   
PIZZERIA                solo somministrazione    0.317181      0.013216   
                        somministrazione/minuto  0.736842      0.000000   
                        NaN                      0.000000      0.000000   
RISTORANTE              solo somministrazione    0.338906      0.014058   
                        somministrazione/minuto  0.568627      0.000000   
                        NaN                      0.000000      0.000000   

Forma vendita                                         NaN  
Tipo esercizio macro pe Forma commercio                    
ALTRO                   solo somministrazione    0.789474  
                        somministrazione/minuto  0.000000  
                        NaN                      0.000000  
BAR                     solo somministrazione    0.203179  
                        somministrazione/minuto  0.004494  
                        NaN                      0.000000  
GASTRONOMIA             solo somministrazione    0.914286  
                        somministrazione/minuto  0.000000  
                        NaN                      0.000000  
GELATERIA               solo somministrazione    0.000000  
                        somministrazione/minuto  0.000000  
                        NaN                      1.000000  
PIZZERIA                solo somministrazione    0.312775  
                        somministrazione/minuto  0.000000  
                        NaN                      0.000000  
RISTORANTE              solo somminist

Forma vendita                                 al banco  al tavolo     misto  \
Tipo esercizio macro pe Forma commercio prev                                  
ALTRO                   minuto                1.000000   0.000000  0.000000   
                        somministrazione      0.150000   0.000000  0.100000   
                        NaN                   0.000000   0.000000  0.000000   
BAR                     minuto                0.770000   0.000000  0.230000   
                        somministrazione      0.478616   0.013506  0.321891   
                        NaN                   0.000000   0.000000  0.000000   
GASTRONOMIA             minuto                1.000000   0.000000  0.000000   
                        somministrazione      0.044898   0.008163  0.028571   
                        NaN                   0.000000   0.000000  0.000000   
GELATERIA               minuto                1.000000   0.000000  0.000000   
                        somministrazione      1.000000   0.000000  0.000000   
                        NaN                   0.000000   0.000000  0.000000   
PIZZERIA                minuto                1.000000   0.000000  0.000000   
                        somministrazione      0.168724   0.172840  0.353909   
                        NaN                   0.000000   0.000000  0.000000   
RISTORANTE              minuto                0.571429   0.142857  0.285714   
                        somministrazione      0.104779   0.365074  0.347794   
                        NaN                   0.000000   0.000000  0.000000   

Forma vendita                                 self service       NaN  
Tipo esercizio macro pe Forma commercio prev                          
ALTRO                   minuto                    0.000000  0.000000  
                        somministrazione          0.000000  0.750000  
                        NaN                       0.000000  0.000000  
BAR                     minuto                    0.000000  0.000000  
                        somministrazione          0.001970  0.184018  
                        NaN                       0.000000  0.000000  
GASTRONOMIA             minuto                    0.000000  0.000000  
                        somministrazione          0.004082  0.914286  
                        NaN                       0.000000  0.000000  
GELATERIA               minuto                    0.000000  0.000000  
                        somministrazione          0.000000  0.000000  
                        NaN                       0.000000  1.000000  
PIZZERIA                minuto                    0.000000  0.000000  
                        somministrazione          0.012346  0.292181  
                        NaN                       0.000000  0.000000  
RISTORANTE              minuto                    0.000000  0.000000  
                        somministrazione          0.013603  0.168750  
                        NaN                       0.000000  0.000000

Ha senso trattare “Forma vendita” come un missing value (e quindi sostituire i NaN con “non dichiarata”) perché dai controlli di correlazione con le macro-categorie e con “Forma commercio/Forma commercio prev” non emerge una regola sufficientemente deterministica e stabile da permettere un’imputazione affidabile: in molte macro-categorie la distribuzione è molto “mista” (nessuna modalità dominante) e in alcuni casi i missing sono addirittura molto concentrati (ad esempio per GASTRONOMIA), segnale che il dato può essere stato semplicemente non raccolto o non compilato in modo sistematico. In questa situazione imputare “al banco/al tavolo/misto” rischierebbe di introdurre valori inventati e distorcere le analisi; invece trasformare il NaN in una categoria esplicita (“non dichiarata”) preserva l’informazione che il dato manca, rende la colonna utilizzabile in conteggi e modelli, e permette anche di analizzare separatamente il fenomeno dei missing senza confonderlo con le categorie reali.

In [34]:
MILANO["Forma vendita_filled"] = MILANO["Forma vendita"].fillna("non dichiarata")
